In [3]:
%cd practicas/pr402/

/media/notebooks/practicas/pr402


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Ejercicio 1

In [5]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue  # saltar cabecera
    line = line.strip().split(",")
    print(f"{line[3]}\t{line[7]}")



Overwriting mapper1.py


In [24]:
%%writefile reducer1.py
#!/usr/bin/env python3

import sys

current_city = None
current_temperature = 0

for line in sys.stdin:
    city, temperature = line.strip().split("\t", 1)
    temperature = float(temperature)
    
    if current_city is None:
        current_city = city
        current_temperature = temperature
    elif city == current_city:
        if temperature > current_temperature:
            current_temperature = temperature
    else:
        print(f"{current_city} ---> {current_temperature}")
        current_city = city
        current_temperature = temperature

if current_city is not None:
    print(f"{current_city} ---> {current_temperature}")


Overwriting reducer1.py


In [16]:
!hdfs dfs -mkdir /pr402

mkdir: `/pr402': File exists


In [17]:
!hdfs dfs -put city_temperature.csv /pr402/

put: `/pr402/city_temperature.csv': File exists


In [26]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper1.py \
-file reducer1.py \
-mapper mapper1.py \
-reducer reducer1.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida1


2025-11-19 11:41:21,834 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper1.py, reducer1.py, /tmp/hadoop-unjar7948557900154159834/] [] /tmp/streamjob7743174051264656020.jar tmpDir=null
2025-11-19 11:41:22,407 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2025-11-19 11:41:22,518 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2025-11-19 11:41:22,690 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763549611594_0006
2025-11-19 11:41:23,080 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-19 11:41:23,094 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.4:9866
2025-11-19 11:41:23,095 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.6:9866
2025-11-19 11:41:23,095 INFO net.Network

In [ ]:
#!hdfs dfs -rm -r /pr402/salida1

Deleted /pr402/salida1


In [27]:
!hdfs dfs -cat /pr402/salida1/part-00000

Abidjan ---> 88.6	
Abilene ---> 94.2	
Abu Dhabi ---> 107.3	
Addis Ababa ---> 77.0	
Akron Canton ---> 86.1	
Albany ---> 88.0	
Albuquerque ---> 89.4	
Algiers ---> 96.6	
Allentown ---> 91.1	
Almaty ---> 90.9	
Amarillo ---> 92.5	
Amman ---> 95.4	
Amsterdam ---> 85.5	
Anchorage ---> 75.3	
Ankara ---> 87.9	
Antananarivo ---> 78.5	
Ashabad ---> 102.2	
Asheville ---> 85.1	
Athens ---> 94.3	
Atlanta ---> 92.8	
Atlantic City ---> 93.3	
Auckland ---> 75.4	
Austin ---> 94.5	
Baltimore ---> 91.9	
Bangkok ---> 93.0	
Bangui ---> 93.7	
Banjul ---> 93.6	
Barcelona ---> 86.6	
Baton Rouge ---> 90.2	
Beijing ---> 92.9	
Beirut ---> 91.1	
Belfast ---> 77.6	
Belgrade ---> 91.9	
Belize City ---> 92.9	
Bern ---> 83.7	
Bilbao ---> 94.6	
Billings ---> 90.6	
Birmingham ---> 91.0	
Bishkek ---> 91.1	
Bismarck ---> 91.7	
Bissau ---> 100.1	
Bogota ---> 66.7	
Boise ---> 94.2	
Bombay (Mumbai) ---> 92.6	
Bonn ---> 86.9	
Bordeaux ---> 88.8	
Boston ---> 90.7	
Brasilia ---> 87.7	
Bratislava ---> 85.5	
Brazzaville ---> 88.7

# Ejercicio 2 

In [1]:
%%writefile mapper2.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(",")
    print(f"{line[1]}\t{line[7]}")

Writing mapper2.py


In [29]:
%%writefile reducer2.py
#!/usr/bin/env python3

import sys

c_pais = None
c_temp = 0
count = 0
total = 0
media = 0


for line in sys.stdin:
    pais, temp = line.strip().split("\t", 1)
    temp = float(temp)

    if c_pais == None:
        c_pais = pais
        c_temp = temp
        count += 1
    elif c_pais == pais:
        c_temp = temp
        count += 1
        total += c_temp
    else:
        media = total / count
        print(f"{c_pais} ---> {media:.2f}")
        c_pais = pais
        c_temp = temp
        media = 0 
        total = c_temp
        count = 1
        
if pais is not None:
    print(f"{pais} ---> {media:.2f}")

Overwriting reducer2.py


In [30]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper2.py \
-file reducer2.py \
-mapper mapper2.py \
-reducer reducer2.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida2

2025-11-19 11:42:16,644 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper2.py, reducer2.py, /tmp/hadoop-unjar7449669868225787628/] [] /tmp/streamjob7926732715513424515.jar tmpDir=null
2025-11-19 11:42:17,179 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2025-11-19 11:42:17,293 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2025-11-19 11:42:17,457 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763549611594_0007
2025-11-19 11:42:17,847 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-19 11:42:17,868 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.3:9866
2025-11-19 11:42:17,868 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.4:9866
2025-11-19 11:42:17,868 INFO net.Network

In [31]:
!hdfs dfs -cat /pr402/salida2/part-00000

Albania ---> 33.17	
Algeria ---> 63.76	
Argentina ---> 62.30	
Australia ---> 61.63	
Austria ---> 51.05	
Bahamas ---> 76.57	
Bahrain ---> 80.64	
Bangladesh ---> 10.11	
Barbados ---> 77.00	
Belarus ---> 41.82	
Belgium ---> 51.06	
Belize ---> 73.48	
Benin ---> 76.15	
Bermuda ---> 66.97	
Bolivia ---> 44.87	
Brazil ---> 70.13	
Bulgaria ---> 45.20	
Burundi ---> -65.40	
Canada ---> 42.01	
Central African Republic ---> 67.02	
China ---> 59.98	
Colombia ---> 55.24	
Congo ---> 69.32	
Costa Rica ---> 70.38	
Croatia ---> 46.93	
Cuba ---> 72.64	
Cyprus ---> 23.78	
Czech Republic ---> 47.60	
Denmark ---> 46.96	
Dominican Republic ---> 65.22	
Egypt ---> 71.95	
Equador ---> 59.95	
Ethiopia ---> 25.46	
Finland ---> 42.24	
France ---> 54.70	
Gabon ---> 70.40	
Gambia ---> 59.67	
Georgia ---> 46.52	
Germany ---> 6.42	
Greece ---> 59.63	
Guatemala ---> 56.91	
Guinea ---> 49.39	
Guinea-Bissau ---> 2.39	
Guyana ---> -22.10	
Haiti ---> 16.69	
Honduras ---> 68.53	
Hong Kong ---> 75.06	
Hungary ---> 51.10	
Icel

In [ ]:
#!hdfs dfs -rm -r /pr402/salida2

Deleted /pr402/salida2


# Ejercicio 3

In [17]:
%%writefile mapper3.py
#!/usr/bin/env python3

import sys

first = True

for line in sys.stdin:
    if first:
        first = False
        continue
    line = line.strip().split(",")
    print(f"{line[3]}\t{line[6]}\t{line[7]}")

Overwriting mapper3.py


In [18]:
%%writefile reducer3.py
#!/usr/bin/env python3

import sys

c_ciudad = None
ciudad = None
c_año = 0
año = 0
c_temp = 0
temp = 0
count = 0

for line in sys.stdin:
    line = line.strip().split("\t", 1)
    ciudad = line[0]
    año = line[1]
    temp = line[2]

    if c_ciudad == None:
        c_ciudad = ciudad
        c_año = año
        if ((temp - 32) * 5 / 9) >= 30:
            count += 1
    elif ciudad == c_ciudad & año == c_año:
        if ((temp - 32) * 5 / 9) >= 30:
            count += 1
    else:
        print(f"{c_ciudad} ({c_año}) ---> {count}")
        c_ciudad = ciudad
        if ((temp - 32) * 5 / 9) >= 30:
            count = 1
        else:
            count = 0

if ciudad is not None:
    print(f"{ciudad} ({c_año}) ---> {count}")

Overwriting reducer3.py


In [21]:
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper3.py \
-file reducer3.py \
-mapper mapper3.py \
-reducer reducer3.py \
-input /pr402/city_temperature.csv \
-output /pr402/salida3

2025-11-19 18:05:53,507 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper3.py, reducer3.py, /tmp/hadoop-unjar1222780068253979877/] [] /tmp/streamjob6352431801035638408.jar tmpDir=null
2025-11-19 18:05:54,125 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-19 18:05:54,246 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.6:8032
2025-11-19 18:05:54,431 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763574026732_0002
2025-11-19 18:05:54,855 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-19 18:05:54,877 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.4:9866
2025-11-19 18:05:54,878 INFO net.NetworkTopology: Adding a new node: /default-rack/172.20.0.2:9866
2025-11-19 18:05:54,878 INFO net.Network

In [20]:
!hdfs dfs -rm -r /pr402/salida3

Deleted /pr402/salida3
